In [1]:
from main.models import Zipcode, BlockGroup, Neighborhood, Listing, Tract
from django.contrib.gis.geos import (
    GEOSGeometry, 
    Polygon, 
    MultiPolygon,
    WKBReader, 
    WKBWriter,
    Point
)
from django.contrib.gis.gdal import DataSource
from django.db.models import ExpressionWrapper, F
import re
from lxml import etree

## Discussion regarding overlap detection of unevenly aligned geo boundaries
http://gis.stackexchange.com/questions/69139/how-do-i-use-polygon-centroids-within-a-geoqueryset

## ZCTA Import

In [ ]:
# Open US Zip Code Tabulation Areas shapefile
ds = DataSource('/Users/shawn/Downloads/cb_2015_us_zcta510_500k/cb_2015_us_zcta510_500k.kml')
layer = ds[0] # There's only 1 layer in this file

In [ ]:
# Create and save zipcode objects
for feature in layer:
    # Extract attrs
    root = etree.fromstring(feature.get('Description'))
    table = root.find('table')
    attrs = {
        tr[0].text: tr[1].text
        for tr in table.findall('tr')[1:]
    }
    geom = GEOSGeometry(feature.geom.json, srid=4326)
    geom = WKBReader().read(wkb=WKBWriter(dim=2).write(geom)) # Coerce 3D input geometry to 2D
    if geom.geom_type != 'MultiPolygon':
        geom = MultiPolygon(geom)

    # Create or update zipcode object
    Zipcode.objects.update_or_create(
        zipcode=attrs['ZCTA5CE10'],
        land_area=float(attrs['ALAND10']),
        water_area=float(attrs['AWATER10']),
        mpoly=geom
    )

## Block Group Import

In [ ]:
# Open Los Angeles County Block Groups shapefile
ds = DataSource('/Users/shawn/Downloads/cb_2015_06_bg_500k/cb_2015_06_bg_500k.kml')
layer = ds[0] # There's only 1 layer in this file

In [ ]:
# Create and save BlockGroup objects
for feature in layer:
    # Extract attributes
    root = etree.fromstring(feature.get('Description'))
    table = root.find('table')
    attrs = {
        tr[0].text: tr[1].text
        for tr in table.findall('tr')[1:]
    }
    
    # Extract geometry
    geom = GEOSGeometry(feature.geom.json, srid=4326)
    geom = WKBReader().read(wkb=WKBWriter(dim=2).write(geom)) # Coerce 3D input geometry to 2D
    if geom.geom_type != 'MultiPolygon':
        geom = MultiPolygon(geom)

    # Create or update object
    BlockGroup.objects.update_or_create(
        geoid=attrs['GEOID'], 
        land_area=float(attrs['ALAND']), 
        water_area=float(attrs['AWATER']),
        mpoly=geom
    )

In [ ]:
# Assign listings to zipcodes
Listing.objects.update(zipcode=None)
for l in Listing.objects.all():
    zipcodes = Zipcode.objects.filter(mpoly__contains_properly=l.point)
    if zipcodes.count() == 1:
        l.zipcode = zipcodes.first()
        l.save()
    else:
        print(l.id, zipcodes, '\n')

In [ ]:
# assign listings to block groups
for l in Listing.objects.all():
    bgs = BlockGroup.objects.filter(mpoly__contains_properly=l.point)
    if bgs.count() == 1:
        l.block_group = bgs.first()
        l.save()
    else:
        print(l.pk, bgs, '\n')

## Tract Import

In [ ]:
# Open Los Angeles County Block Groups shapefile
ds = DataSource('../res/cb_2015_06_tract_500k.kml')
layer = ds[0] # There's only 1 layer in this file

In [ ]:
# Create and save Tract objects
for feature in layer:
    # Extract attributes
    root = etree.fromstring(feature.get('Description'))
    table = root.find('table')
    attrs = {
        tr[0].text: tr[1].text
        for tr in table.findall('tr')[1:]
    }
    
    # Extract geometry
    geom = GEOSGeometry(feature.geom.json, srid=4326)
    geom = WKBReader().read(wkb=WKBWriter(dim=2).write(geom)) # Coerce 3D input geometry to 2D
    if geom.geom_type != 'MultiPolygon':
        geom = MultiPolygon(geom)

    # Create or update object
    Tract.objects.update_or_create(
        geoid=attrs['GEOID'], 
        land_area=float(attrs['ALAND']), 
        water_area=float(attrs['AWATER']),
        mpoly=geom
    )

In [ ]:
# Set centroids
for t in Tract.objects.all():
    t.centroid = t.mpoly.centroid
    t.save()


In [2]:
# Set tract-->neighborhood relations
for n in Neighborhood.objects.all():
    Tract.objects.filter(centroid__within=n.mpoly).update(neighborhood=n)

# Set tract-->listing relations
for t in Tract.objects.all():
    Listing.objects.filter(point__within=t.mpoly).update(tract=t)

In [3]:
Listing.objects.filter(tract=None).count()

20

In [5]:
Tract.objects.exclude(neighborhood=None).count()

2340